In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Code adapted from riken_demo.m in https://github.com/FieteLab/neural-circuit-inference/tree/main

Values for recurrent weight ($r$) and spiking threshold are obtained from thresholds_pinned.mat in the repo above

In [ ]:
# values for r and spiking threshold 
all_rs = [0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02, 0.0225, 0.025]
all_threshs = [0.0013, 0.0012, 0.0011, 0.001, 9.55e-4, 8.93e-4, 8.52e-4, 8.15e-4, 7.75e-4, 7.4e-4]

In [ ]:
# PARAMETERS
dt = 0.0001         # time-step in seconds
N = 100             # number of neurons
tau = 0.01          # synaptic time-constant in seconds

# Weight parameters
sig_1 = 6.98
sig_2 = 7
a1 = 1
a2 = 1.0005

# Create weight matrix W
indices = np.arange(N)
i_indices = indices.reshape(N, 1)
j_indices = indices.reshape(1, N)
x = np.minimum(np.abs(i_indices - j_indices), N - np.abs(i_indices - j_indices))
W = a1 * (np.exp(-x**2 / (2 * sig_1**2)) - a2 * np.exp(-x**2 / (2 * sig_2**2)))

plt.imshow(W)
plt.colorbar()
plt.title('Weight matrix W')
plt.show()

In [ ]:
W.shape

In [ ]:
np.save('W.npy', W)

In [ ]:
# Other parameters
b = 0.001           # uniform feedforward input
noise_sd = 0.3      # amplitude of noise
noise_sparsity = 1.5  # noise is injected with the probability that a standard normal exceeds this
r = 0.005           # recurrence strength
threshold = 0.0012 # spiking threshold

In [ ]:
start_rec = 1 # sec. start collecting spikes
start_t = int(start_rec/dt)
print(start_t)

In [ ]:
rng = np.random.default_rng(seed=0)
s = np.zeros((N, 1))     # starting activations
t = 0

total_time = 1000000

activation = []
all_spikes = []

while t < total_time:
    # Dynamics
    noise = noise_sd * rng.standard_normal(size=(N, 1)) * (rng.standard_normal(size=(N, 1)) > noise_sparsity)
    I = r * W @ s + b * (1 + noise)  # neuron inputs
    spike = I > threshold                    # binary neural spikes
    s = s + spike.astype(float) - s / tau * dt  # update activations
    activation.append(s)
    all_spikes.append(spike)
    t += 1

In [ ]:
activation = np.array(activation).squeeze()
all_spikes = np.array(all_spikes).squeeze()

In [ ]:
activation = activation.astype(np.float32)

In [ ]:
activation.shape, all_spikes.shape

In [ ]:
# number of spikes in the collection window
np.sum(all_spikes[start_t:, :])

In [ ]:
r, threshold

In [ ]:
np.save(f'activation_r{r}.npy', activation)

In [ ]:
np.save(f'spikes_r{r}.npy', all_spikes)